In [143]:
import numpy as np
import cv2
import tensorflow as tf
import pickle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D,Dense,Activation,Flatten,MaxPooling2D,Input,Lambda,Dropout,Conv1D
from tensorflow.keras.models import Model,load_model

In [121]:
cifar=load_model('cifar1.h5')

In [122]:
cifar.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 128)       0   

In [76]:
data=np.load('C:\\Users\\YASH\\Cifar-10\\cifar_data.npy')
train_x=data[0:55000]
test_x=data[55000:]
label=np.load('C:\\Users\\YASH\\Cifar-10\\cifar_label_onehot.npy')
train_label=label[0:55000]
test_label=label[55000:]

In [82]:
class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self,epochs,cd):
        count=0
        count2=0
        norm={}
        threshold=0.019
#         print(epochs)
#         print(cd)
        for layers in self.model.layers:
            if 'conv2d' in layers.name :#or 'dense' in layers.name:
                a=self.model.get_layer(layers.name)
                threshold = threshold - 0.002
                e=a.get_weights()
                if(len(e)>0):
                    weights1 = np.asarray(e[0])
                    bias1 = np.asarray(e[1])
                    dim = weights1.shape[0]*weights1.shape[1]*weights1.shape[2]
                    squared_sum = (np.linalg.norm(np.linalg.norm(weights1,axis=(0,1)),axis=0)**2)/dim
                    upper_threshold = (squared_sum>threshold)
                    lower_threshold = (squared_sum<-threshold)
                    weights1 = weights1*(upper_threshold+lower_threshold)
                    bias1 = bias1*(upper_threshold+lower_threshold)
                    count = np.sum((upper_threshold+lower_threshold)==False)*dim+count
                    count2 = np.sum((upper_threshold+lower_threshold)==True)*dim+count2
                    e[0] = weights1
                    e[1] = bias1
                    a.set_weights(e)
        print('sparsity: '+str((count/(count2+count))*100))
        
                

In [124]:
cifar=load_model('cifar1.h5')
count=0
count2=0
norm={}
threshold=0.005
for layers in cifar.layers:
    if 'dense' or 'conv2d' or 'conv1d' in layers.name:
        a=cifar.get_layer(layers.name)
        e=a.get_weights()
        if(len(e)>0):
            dim = np.prod(e[0].shape[:-1])
            squared_sum = np.average(e[0]**2,axis=tuple(np.arange(0,len(e[0].shape)-1)))
            upper_threshold = (squared_sum>threshold)
            e[0] = e[0]*(upper_threshold)
            e[1] = e[1]*(upper_threshold)
            print(np.sum(upper_threshold))
            a.set_weights(e)
            print(layers.name)
            count = np.sum((upper_threshold)==False)*dim+count
            count2 = np.sum((upper_threshold)==True)*dim+count2
print('sparsity: '+str((count/(count2+count))*100))
        
                

63
conv2d_10
50
conv2d_11
112
conv2d_12
153
conv2d_13
178
conv2d_14
361
conv2d_15
201
conv2d_16
104
conv2d_17
2
conv2d_18
57
dense_4
128
dense_5
10
dense_6
sparsity: 68.14419297409923


In [125]:
cifar=load_model('cifar1.h5')
count=0
count2=0
norm={}
threshold=0.005
for layers in cifar.layers:
    if 'dense' in layers.name:
        a=cifar.get_layer(layers.name)
#         threshold = threshold - 0.002
        e=a.get_weights()
        if(len(e)>0):
            dim = np.prod(e[0].shape[:-1])
            squared_sum = np.average(e[0]**2,axis=tuple(np.arange(0,len(e[0].shape)-1)))
            upper_threshold = (squared_sum>threshold)
            e[0] = e[0]*(upper_threshold)
            e[1] = e[1]*(upper_threshold)
            print(np.sum(upper_threshold))
            a.set_weights(e)
            print(layers.name)
            count = np.sum((upper_threshold)==False)*dim+count
            count2 = np.sum((upper_threshold)==True)*dim+count2
print('sparsity: '+str((count/(count2+count))*100))
        
                

57
dense_4
128
dense_5
10
dense_6
sparsity: 43.69230769230769


In [126]:
cifar=load_model('cifar1.h5')
count=0
count2=0
norm={}
threshold=0.005
for layers in cifar.layers:
    if 'conv2d' in layers.name:
        a=cifar.get_layer(layers.name)
#         threshold = threshold - 0.002
        e=a.get_weights()
        if(len(e)>0):
            dim = np.prod(e[0].shape[:-1])
            squared_sum = np.average(e[0]**2,axis=tuple(np.arange(0,len(e[0].shape)-1)))
            upper_threshold = (squared_sum>threshold)
            e[0] = e[0]*(upper_threshold)
            e[1] = e[1]*(upper_threshold)
            print(np.sum(upper_threshold))
            a.set_weights(e)
            print(layers.name)
            count = np.sum((upper_threshold)==False)*dim+count
            count2 = np.sum((upper_threshold)==True)*dim+count2
print('sparsity: '+str((count/(count2+count))*100))
        
                

63
conv2d_10
50
conv2d_11
112
conv2d_12
153
conv2d_13
178
conv2d_14
361
conv2d_15
201
conv2d_16
104
conv2d_17
2
conv2d_18
sparsity: 68.36401864069211


In [127]:
cifar=load_model('cifar1.h5')
count=0
count2=0
norm={}
threshold=0.005
for layers in cifar.layers:
    if 'dense' or 'conv2d' in layers.name:
        a=cifar.get_layer(layers.name)
#         threshold = threshold - 0.002
        e=a.get_weights()
        if(len(e)>0):
            dim = np.prod(e[0].shape[:-1])
            squared_sum = np.average(e[0]**2,axis=tuple(np.arange(0,len(e[0].shape)-1)))
            upper_threshold = (squared_sum>threshold)
            e[0] = e[0]*(upper_threshold)
            e[1] = e[1]*(upper_threshold)
            print(np.sum(upper_threshold))
            a.set_weights(e)
            print(layers.name)
            count = np.sum((upper_threshold)==False)*dim+count
            count2 = np.sum((upper_threshold)==True)*dim+count2
print('sparsity: '+str((count/(count2+count))*100))
        
                

63
conv2d_10
50
conv2d_11
112
conv2d_12
153
conv2d_13
178
conv2d_14
361
conv2d_15
201
conv2d_16
104
conv2d_17
2
conv2d_18
57
dense_4
128
dense_5
10
dense_6
sparsity: 68.14419297409923


In [128]:
cifar=load_model('cifar1.h5')
count=0
count2=0
norm={}
threshold=0.005
#         print(epochs)
#         print(cd)
for layers in cifar.layers:
    if 'conv2d' in layers.name :#or 'dense' in layers.name:
        a=cifar.get_layer(layers.name)
#         threshold = threshold - 0.002
        e=a.get_weights()
        if(len(e)>0):
            weights1 = np.asarray(e[0])
            bias1 = np.asarray(e[1])
            dim = weights1.shape[0]*weights1.shape[1]*weights1.shape[2]
            squared_sum = (np.linalg.norm(np.linalg.norm(weights1,axis=(0,1)),axis=0)**2)/dim
            upper_threshold = (squared_sum>threshold)
            lower_threshold = (squared_sum<-threshold)
            weights1 = weights1*(upper_threshold+lower_threshold)
            bias1 = bias1*(upper_threshold+lower_threshold)
            print(np.sum(upper_threshold))
            count = np.sum((upper_threshold+lower_threshold)==False)*dim+count
            count2 = np.sum((upper_threshold+lower_threshold)==True)*dim+count2
            e[0] = weights1
            e[1] = bias1
            a.set_weights(e)
print('sparsity: '+str((count/(count2+count))*100))



63
50
112
153
178
361
201
104
2
sparsity: 68.36401864069211


In [33]:
np.prod(weights1.shape[:-1])

2304

In [34]:
weights1.shape[:-1]

(3, 3, 256)

In [70]:
np.average(weights1**2,axis=tuple(np.arange(0,len(weights1.shape)-1))).shape

(256,)

In [64]:
tuple((np.arange(2,5)))

(2, 3, 4)

In [68]:
len(weights1.shape)

4

In [81]:

count=0
count2=0
norm={}
threshold=0.019
#         print(epochs)
#         print(cd)
for layers in cifar.layers:
    if 'conv2d' in layers.name :#or 'dense' in layers.name:
        a=cifar.get_layer(layers.name)
        threshold = threshold - 0.002
        e=a.get_weights()
        if(len(e)>0):
            weights1 = np.asarray(e[0])
            bias1 = np.asarray(e[1])
            dim = weights1.shape[0]*weights1.shape[1]*weights1.shape[2]
            squared_sum = (np.linalg.norm(np.linalg.norm(weights1,axis=(0,1)),axis=0)**2)/dim
            upper_threshold = (squared_sum>threshold)
            lower_threshold = (squared_sum<-threshold)
            weights1 = weights1*(upper_threshold+lower_threshold)
            bias1 = bias1*(upper_threshold+lower_threshold)
            count = np.sum((upper_threshold+lower_threshold)==False)*dim+count
            count2 = np.sum((upper_threshold+lower_threshold)==True)*dim+count2
            e[0] = weights1
            e[1] = bias1
            a.set_weights(e)
print('sparsity: '+str((count/(count2+count))*100))

sparsity: 55.03954145142217


In [ ]:
# def on_train_end():
delete_ind=[]
layer_details={}
allow=0
from keras.models import model_from_json
import json
prunned_model = cifar.to_json()
res = json.loads(prunned_model) 
i=0
for layers in cifar.layers:
    a=cifar.get_layer(layers.name)
    e=a.get_weights()
    if 'flatten' in layers.name:
        allow=1
    if len(e)>0:    # avoid input, maxpooling layers
        weights = np.asarray(e[0])
        bias = np.asarray(e[1])
        if 'conv2d' in layers.name:
            weights = np.delete(weights,delete_ind,2)
            squared_sum = np.linalg.norm(np.linalg.norm(weights,axis=(0,1)),axis=0)
            delete_ind = np.nonzero(squared_sum==0)
            weights = np.delete(weights,delete_ind,3)
            bias = np.delete(bias,delete_ind,0)
            if (layers.name==res['config']['layers'][i]['name']):
                res['config']['layers'][i]['config']['filters']=weights.shape[3]                
                
        if 'dense' in layers.name and allow == 1:
            weights = np.delete(weights,delete_ind,0)
            allow = 0
        layer_details[layers.name] = {'shape': weights.shape,'weights':weights,'bias':bias}
        print(weights.shape)

    i=i+1
res=json.dumps(res)
prunned_model = tf.keras.models.model_from_json(res)
for layers in prunned_model.layers:
    if 'conv2d' in layers.name or 'dense' in layers.name :#or 'dense' in layers.name:
        a=prunned_model.get_layer(layers.name)
        e=a.get_weights()
        e[0] = layer_details[layers.name]['weights']
        e[1] = layer_details[layers.name]['bias']
        a.set_weights(e)        
        
prunned_model.summary()

In [ ]:
layer_details['conv2d_55']['weights']

In [ ]:
layer_details

In [ ]:
from keras.models import model_from_json
json_model = cifar.to_json()

In [ ]:
res = json.loads(json_model) 

In [ ]:
res['config']['layers'][2]['config']['filters']

In [ ]:
for layers in cifar.layers:
    print(layers.name)

In [ ]:
cifar=load_model('cifar1.h5')

In [ ]:
model_j = tf.keras.models.model_from_json(res)

In [ ]:
model_j.summary()

In [ ]:
model_j = tf.keras.models.model_from_json(json_model)

In [ ]:
prunned_model.summary()

In [ ]:
opt=Adam(lr=0.0001/2)
cifar.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
cifar.fit(train_x,train_label,epochs=1,validation_split=0.05,batch_size=64)#,callbacks=[MyCustomCallback()])

In [ ]:
prunned_model.save('befquan.h5')

In [ ]:
z=np.linalg.norm(np.linalg.norm(c,axis=(0,1)),axis=0)/(c.shape[0]+c.shape[1]+c.shape[2])
z.shape

In [ ]:
opt=Adam(lr=0.001)
quan_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
prunned_model.evaluate(x=test_x,y=test_label)

In [ ]:
quan_model.evaluate(x=test_x,y=test_label)

In [12]:
cifar.evaluate(x=test_x,y=test_label)

5000/5000 [==============================] - 2s 452us/sample - loss: 0.5680 - accuracy: 0.8320


[0.568046052646637, 0.832]

In [ ]:
cifar.evaluate(x=train_x,y=train_label)

In [5]:
import json
quan_model = cifar.to_json()
res = json.loads(quan_model) 

In [ ]:
for i in range(0,22):
    res['config']['layers'][i]['config']['dtype']='float16'

In [ ]:
res=json.dumps(res)
quan_model = tf.keras.models.model_from_json(res)
for layers in quan_model.layers:
    if 'conv2d' in layers.name or 'dense' in layers.name :#or 'dense' in layers.name:
        a=quan_model.get_layer(layers.name)
        e=a.get_weights()
        e[0] = layer_details[layers.name]['weights']
        e[1] = layer_details[layers.name]['bias']
        a.set_weights(e)        
        
quan_model.summary()

In [ ]:
quan123_model = quan_model.to_json()
res = json.loads(quan123_model) 

In [ ]:
opt=Adam(lr=0.001)
quan_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
quan_model.evaluate(x=test_x,y=test_label)

In [ ]:
quan_model.save('aftquan.h5')

In [134]:
a=cifar.get_layer('conv2d_15')
e=a.get_weights()

In [135]:
e[0].shape

(3, 3, 256, 512)

In [ ]:
c[0,0,1,1]

In [136]:
a=cifar.get_layer('dense_6')
e=a.get_weights()

In [141]:
e[0].shape[-2]

128

In [142]:
len(e[0].shape)

2

In [6]:
res

{'class_name': 'Model',
 'config': {'name': 'model_2',
  'layers': [{'class_name': 'InputLayer',
    'config': {'batch_input_shape': [None, 32, 32, 3],
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'input_2'},
    'name': 'input_2',
    'inbound_nodes': []},
   {'class_name': 'Lambda',
    'config': {'name': 'lambda_2',
     'trainable': True,
     'dtype': 'float32',
     'function': ['4wEAAAAAAAAAAQAAAAIAAABDAAAAcwwAAAB8AGQBGwBkAhgAUwApA07p/wAAAGcAAAAAAADgP6kA\nKQHaAXhyAgAAAHICAAAA+h48aXB5dGhvbi1pbnB1dC05LTFhZTE4OWM4NzEwMT7aCDxsYW1iZGE+\nAQAAAPMAAAAA\n',
      None,
      None],
     'function_type': 'lambda',
     'module': 'tensorflow.python.keras.layers.core',
     'output_shape': None,
     'output_shape_type': 'raw',
     'output_shape_module': None,
     'arguments': {}},
    'name': 'lambda_2',
    'inbound_nodes': [[['input_2', 0, 0, {}]]]},
   {'class_name': 'Conv2D',
    'config': {'name': 'conv2d_10',
     'trainable': True,
     'dtype'

In [145]:
inp=Input((128,128))
x=Conv1D(100,3)(inp)
x=Conv1D(50,5)(x)

In [146]:
mod=Model(inp,x)
mod.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128)]        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 126, 100)          38500     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 122, 50)           25050     
Total params: 63,550
Trainable params: 63,550
Non-trainable params: 0
_________________________________________________________________


In [155]:
a=mod.get_layer('conv1d_2')
b=a.get_weights()

In [156]:
b[0].shape

(5, 100, 50)

In [ ]:
# Final compression
delete_ind=[]
layer_details={}
allow=0
from keras.models import model_from_json
import json
prunned_model = rnn.to_json() # converting model into json

res = json.loads(prunned_model) 
i=0

for layers in rnn.layers:
    a=rnn.get_layer(layers.name) # getting weights
    e=a.get_weights()
    
    #checking for layers with none parameters
    if len(e)>0:
        if 'lstm' in layers.name:  # for lstm layer 
            e[0] = np.delete(e[0],delete_ind,0)  # removing dimension from previous layers 
            avg =np.average(e[1]**2,axis=0)+np.average(e[0]**2,axis=0) # avg to find which filters to remove
            weights1=0
            units = e[1].shape[0]
            
            for j in range(0,4):
                weights1=weights1+avg[units*j:units*(j+1)]
            delete_ind = np.nonzero(weights1==0)
            
            #removing weights 
            c= np.concatenate((delete_ind[0],delete_ind[0]+units,delete_ind[0]+units*2,delete_ind[0]+units*3))
            e[0] = np.delete(e[0],c,1)
            e[1] = np.delete(e[1],delete_ind,0)
            e[1] = np.delete(e[1],c,1)
            e[2] = np.delete(e[2],c,0)
            
            # changing number of filters in model
            if (layers.name==res['config']['layers'][i]['name']):
                res['config']['layers'][i]['config']['units']=e[1].shape[0]
                
            # repeating for the dense layer 
        if 'dense' in layers.name or 'conv2d' in layers.name or 'conv1d' in layers.name:
            e[0] = np.delete(e[0],delete_ind,(len(e[0].shape)-2))
            avg = np.average(e[0]**2,axis=tuple(np.arange(0,len(e[0].shape)-1)))
            delete_ind = np.nonzero(avg==0)
            e[0] = np.delete(e[0],delete_ind,(len(e[0].shape)-1))
            e[1] = np.delete(e[1],delete_ind,0)
            if (layers.name==res['config']['layers'][i]['name']):
                res['config']['layers'][i]['config']['units']=e[1].shape[0]
            
        layer_details[layers.name] = {'weights':e}
    i=i+1
res=json.dumps(res)
prunned_model = tf.keras.models.model_from_json(res)
for layers in prunned_model.layers:
    if 'lstm' in layers.name or 'dense' in layers.name:
        a=prunned_model.get_layer(layers.name)
        e=a.get_weights()
        e=layer_details[layers.name]['weights']
        a.set_weights(e)        
        
prunned_model.summary()